In [1]:
from pathlib import Path

import numpy as np
from PIL import Image

from sneakers_ml.features.features import load_features
from sneakers_ml.features.hog import get_hog
from sneakers_ml.features.resnet152 import get_resnet152_feature
from sneakers_ml.models.onnx import load_catboost_onnx, load_sklearn_onnx, predict_catboost_onnx, predict_sklearn_onnx

In [2]:
list(Path("data/training/brands-classification-splits/test").iterdir())

[PosixPath('data/training/brands-classification-splits/test/adidas'),
 PosixPath('data/training/brands-classification-splits/test/nike'),
 PosixPath('data/training/brands-classification-splits/test/vans'),
 PosixPath('data/training/brands-classification-splits/test/jordan'),
 PosixPath('data/training/brands-classification-splits/test/saucony'),
 PosixPath('data/training/brands-classification-splits/test/kangaroos'),
 PosixPath('data/training/brands-classification-splits/test/converse'),
 PosixPath('data/training/brands-classification-splits/test/clarks'),
 PosixPath('data/training/brands-classification-splits/test/puma'),
 PosixPath('data/training/brands-classification-splits/test/new balance'),
 PosixPath('data/training/brands-classification-splits/test/reebok'),
 PosixPath('data/training/brands-classification-splits/test/karhu'),
 PosixPath('data/training/brands-classification-splits/test/asics')]

In [4]:
sklearn_hog_models = [
    "data/models/brands-classification/hog-sgd.onnx",
    "data/models/brands-classification/hog-svc.onnx",
]
sklearn_resnet_models = [
    "data/models/brands-classification/resnet-sgd.onnx",
    "data/models/brands-classification/resnet-svc.onnx",
]
catboost_hog_models = ["data/models/brands-classification/hog-catboost.onnx"]
catboost_resnet_models = ["data/models/brands-classification/resnet-catboost.onnx"]

In [5]:
models = {}

for model in sklearn_hog_models + sklearn_resnet_models:
    models[Path(model).stem] = load_sklearn_onnx(model)

for model in catboost_hog_models + catboost_resnet_models:
    models[Path(model).stem] = load_catboost_onnx(model)

In [6]:
def predict_using_all_models(image: Image.Image) -> dict:
    preds = {}
    hog_embedding = get_hog(image)[np.newaxis]
    resnet_embedding = get_resnet152_feature(image)[np.newaxis]
    for model in models:
        print(model)
        if "hog" in model:
            if "catboost" in model:
                preds[model] = predict_catboost_onnx(models[model], hog_embedding)[0][0]
            else:
                preds[model] = predict_sklearn_onnx(models[model], hog_embedding)[0]
        else:
            if "catboost" in model:
                preds[model] = predict_catboost_onnx(models[model], resnet_embedding)[0][0]
            else:
                preds[model] = predict_sklearn_onnx(models[model], resnet_embedding)[0]
    return preds

In [7]:
models

{'hog-sgd': <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession at 0x7fe81c968910>,
 'hog-svc': <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession at 0x7fe81f0417f0>,
 'resnet-sgd': <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession at 0x7fe9181dc760>,
 'resnet-svc': <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession at 0x7fe81c968280>,
 'hog-catboost': <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession at 0x7fe81c9682b0>,
 'resnet-catboost': <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession at 0x7fe9180c2460>}

In [8]:
image = Image.open("data/training/brands-classification-splits/test/asics/6.jpeg")

In [9]:
predict_using_all_models(image)

hog-sgd
hog-svc
resnet-sgd
resnet-svc
hog-catboost
resnet-catboost


{'hog-sgd': '1',
 'hog-svc': '1',
 'resnet-sgd': '1',
 'resnet-svc': '1',
 'hog-catboost': '1',
 'resnet-catboost': '8'}

In [10]:
_, _, class_2_idx = load_features("data/features/brands-classification-splits/hog-train.pickle")
class_2_idx

{'adidas': 0,
 'asics': 1,
 'clarks': 2,
 'converse': 3,
 'jordan': 4,
 'kangaroos': 5,
 'karhu': 6,
 'new balance': 7,
 'nike': 8,
 'puma': 9,
 'reebok': 10,
 'saucony': 11,
 'vans': 12}